In [6]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
import torch
import random
import pandas as pd
import numpy as np
import scipy.io as scp
import torch.optim as optim
import torchvision.models as models
from torchvision.transforms import functional as F

In [7]:
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                        [0.229, 0.224, 0.225])])

testval_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.Flowers102(root='./data', split='train', download=True, transform=train_transform)
val_dataset = torchvision.datasets.Flowers102(root='./data', split='val', download=True, transform=testval_transform)
test_dataset = torchvision.datasets.Flowers102(root='./data', split='test', download=True, transform=testval_transform)

# CutMix

> **Cut-and-Paste Data Augmentation**: CutMix combines two or more images by cutting a rectangular portion from one image and pasting it onto another. The pixel values of the pasted region are a combination of the original image and the selected portion from another image.
<br>
>
> **Label Mixing**: The labels of the pasted region are also combined based on the area. This means that if 60% of the region comes from image A and 40% from image B, the label for that region is mixed accordingly.
<br>
>
> **Smooth Regularization**: CutMix acts as a regularization technique to prevent overfitting. It encourages the model to make predictions on the mixed regions, which can lead to improved generalization.
<br>
>
> **Benefits**: CutMix can improve model robustness, make the model less sensitive to input perturbations, and lead to better generalization. It is especially useful when training on smaller datasets.
<br>
>
# MixUp

> **Linear Interpolation**: MixUp operates by linearly interpolating between pairs of input samples. Given two input samples and their corresponding labels, MixUp creates new training examples by taking a weighted sum of the two samples. The labels are also linearly interpolated.
<br>
>
> **Smooth Labeling**: MixUp effectively "softens" the labels by blending them. For example, if you mix two images with labels "cat" and "dog" with a mixing factor of 0.7, the new image's label will be a soft label with 70% "cat" and 30% "dog."
<br>
>
> **Benefits**: MixUp encourages the model to make predictions that are linear combinations of the original data points. It helps the model learn a more generalized decision boundary and reduce the risk of overfitting. It also aids in handling class imbalance.
<br>
>

In [11]:
def cutmix(data, target, alpha=1.0):
    indices = torch.randperm(data.size(0))
    lam = np.random.beta(alpha, alpha)
    data = data * lam + data[indices] * (1 - lam)
    target = target * lam + target[indices] * (1 - lam)
    return data, target

def mixup(data, target, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    indices = torch.randperm(data.size(0))
    data = data * lam + data[indices] * (1 - lam)
    target = target * lam + target[indices] * (1 - lam)
    return data, target

In [44]:
def train(train_loader, model, criterion, optimizer, device, cutmix_prob=0.5, mixup_prob=0.5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device).float(), targets.to(device).float()
        
        if random.random() < cutmix_prob:
            inputs, targets = cutmix(inputs, targets)
        elif random.random() < mixup_prob:
            inputs, targets = mixup(inputs, targets)
        
        targets = targets.float()
        inputs = inputs.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return running_loss / len(train_loader), correct / total

In [45]:
def eval(dataloader, model, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            pred = output.argmax(dim=1)

            correct += pred.eq(target.view_as(pred)).sum().item() # compare predicted label to actual label
    return correct / len(dataloader.dataset)

In [46]:
def nn_setup(dropout=0.5, hidden_layer1 = 120,lr = 0.001):
    
    model = models.vgg16(pretrained=True)  
        
    for param in model.parameters():
        param.requires_grad = False

        from collections import OrderedDict
        classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 500)),
                          ('relu', nn.ReLU()),
                          ('dropout1', nn.Dropout(dropout)),
                          ('fc2', nn.Linear(500, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
        
        model.classifier = classifier
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.classifier.parameters(), lr )
        
        return model , optimizer ,criterion

In [47]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCH = 100
NUM_CLASSES = 5

# HYPERPARAMS TO TUNE
NUM_HIDDEN = 128
NUM_LAYERS = 1
BATCH_SIZE = 128
EARLY_STOP_THRESHOLD = 3
LR = 0.001
loss_list = []
accuracy_list = []
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
model,optimizer,criterion = nn_setup()
model.to(DEVICE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
best_acc = 0
early_stop_count = 0

In [48]:
for epoch in range(1, NUM_EPOCH+1):
    train_loss = train(train_loader, model, criterion, optimizer, DEVICE)
    accuracy = eval(val_loader, model, DEVICE)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Val Accuracy: {accuracy}')
    if accuracy > best_acc:
        best_acc = accuracy
        early_stop_count = 0
    else:
        early_stop_count += 1
    if early_stop_count >= EARLY_STOP_THRESHOLD:
        print("Early Stopping...")        
        break
    scheduler.step()
test_accuracy = eval(test_loader, model, DEVICE)
print(f'Test Accuracy: {test_accuracy}')

KeyboardInterrupt: 